# Simple Dataset Labeler

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import asyncio
import pickle

In [ ]:
# Example list of prompts.
# Replace with your own list of strings
text_list = ["give me advice","what is the capital of Toronto?","Give me the recipe of pancakes."]

In [ ]:
# Text below heavily inspired by 
# https://stackoverflow.com/questions/55244865/pause-jupyter-notebook-widgets-waiting-for-user-input

def wait_for_change(widget):
    future = asyncio.Future()
    def getvalue(change):
        future.set_result(w.value)
        widget.on_click(getvalue, remove=True) 
        # we need to free up the binding to getvalue to avoid an InvalidState error
        # buttons don't support unobserve
        # so use `remove=True` 
    widget.on_click(getvalue)
    return future

w = widgets.RadioButtons(
options=['instruction', 'fulfilment', 'neither',"unknown"],
value=None,
description='Label:',
disabled=False)

button = widgets.Button(
description='Next',
disabled=False,
tooltip='Click me',
icon='check')

async def f():
    
    # Get saved list or create an empty dict
    try:
        with open('text_and_labels.pickle', 'rb') as handle:
            text_and_labels = pickle.load(handle)
    except:
        text_and_labels = {}
        
    print(f"Total Count of Prompts in Text List: {len(text_list)}")
    print("---------------------------------")
    
    # Loop through list of text
    for i,t in enumerate(text_list):
        
        # Skip items already in the saved file
        if i not in text_and_labels:
            
            # This code prints text on the same line
            print("\r",f"{i}: {t}", end="")
            
            # Await next button click
            x = await wait_for_change(button)
            
            # Add text and label to dict
            text_and_labels[i] = {"text":t,"label":x}

            # Save output
            with open('text_and_labels.pickle', 'wb') as handle:
                pickle.dump(text_and_labels, handle, protocol=-1)
            
    print("")
    print("---------------------------------")
    print("Done")
    return text_and_labels

text_and_labels = asyncio.create_task(f())
display(w,button)

In [ ]:
# Check output
text_and_labels.result()